In [4]:
import dill as pkl
from torch.utils.data import Dataset, DataLoader
from datasets import cropImage

# Notebook to create predictions for the challenge

It uses a pickled testset and an already trained model to make predictions.
The predictions are then packaged for submission.

In [3]:
testSetPath = 'challenge_testset.pkl'
with open(testSetPath,'rb') as tsp:
    testSet = pkl.load(tsp)

In [8]:
testSet.keys()

dict_keys(['images', 'crop_sizes', 'crop_centers'])

In [15]:
testSet['crop_centers'][0]

(30, 25)

In [20]:
int(testSet['crop_sizes'][0][0]/2)

7

In [21]:
top = testSet['crop_centers'][0][0] - int(testSet['crop_sizes'][0][0]/2)
bottom 

23

In [ ]:
class CroppedOutImageDataSet(Dataset):
    def __init__(self, pathToPkl):
      self.pklPath = pathToPkl
      with open(testSetPath,'rb') as tsp:
            testSet = pkl.load(tsp)
      self.images = testSet['images']
      self.crop_sizes = testSet['crop_sizes']
      self.crop_centers = testSet['crop_centers'] 

    def __len__(self):
        return len(self.images)

    def __getitem__(self, id):
        image_data = self.iamges[id]
        height, width = image_data.shape

        image_data = np.asarray(image_data, dtype=np.float32)
        mean = image_data.mean()
        std = image_data.std()
        image_data[:] -= mean
        image_data[:] /= std

        # setting some random crops
        top = self.crop_centers[id][0] - int(self.crop_sizes[0]/2)
        bottom = self.crop_centers[id][0] + int(self.crop_sizes[0]/2)
        left = self.crop_centers[id][1] - int(self.crop_sizes[1]/2)
        right = self.crop_centers[id][1] + int(self.crop_sizes[1]/2)

        cropped_image, crop_array, target_array = cropImage(image_data, top, bottom, left, right)

        # constructing the image we want to input to the model
        inputs = np.zeros(shape = (*cropped_image.shape, 4), dtype=cropped_image.dtype)
        inputs[..., 0] = cropped_image
        inputs[..., 1] = crop_array
        inputs[..., 2] = np.tile(np.linspace(start=-1, stop=1, num=width),reps=[height,1])
        inputs[..., 3] = np.tile(np.expand_dims(np.linspace(start=-1, stop=1, num=height),1),reps=[1,width])

        return TF.to_tensor(inputs), TF.to_tensor(target_array), id